In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import numpy as np
import osmnx as ox
import networkx as nx
import geopandas as gpd
import random
import pandas as pd
import pickle
import os
import pyproj
from shapely.geometry import Point, LineString
from datetime import datetime, timedelta
import ast
from geopy.distance import geodesic
from polyline import decode as decode_polyline
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

# Z-score normalization
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix

from datetime import date

today_date = date.today()
formatted_date = today_date.strftime("%Y-%m-%d")
prior_day = today_date - timedelta(days=1)

scaler = MinMaxScaler()
# Initialize a LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb


In [6]:
df_ml = pd.read_csv('C:\\Users\\Camelia\\Desktop\\app\\Disertatie\\ML\\MLDatasetUntil_'+(today_date-timedelta(days=1)).strftime("%Y-%m-%d")+'.csv')

congestion_mapping = {
    'Low Congestion': 0,
    'Moderate Congestion': 1,
    'High Congestion': 2
}
day_mapping = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}
weather_mapping={
'scattered clouds': 0,
'broken clouds': 1,
'overcast clouds': 2,
'light rain': 3,
'moderate rain' : 4,
'heavy intensity rain': 5,
'few clouds': 6,
'clear sky':7

}
# Map the congestion levels

df_ml['congestion_level'] = df_ml['congestion_level'].map(congestion_mapping)
df_ml['day_of_week'] = df_ml['day_of_week'].map(day_mapping)
df_ml['weather_data'] = df_ml['weather_data'].map(weather_mapping)
#df_ml['congestion_level'] = label_encoder.fit_transform(df_ml['congestion_level'].values)
X = df_ml.drop(['congestion_level','timestamp','origin_osmid','destination_osmid','Unnamed: 0'], axis=1)  # Features are all columns except 'congestion_level'
y = df_ml['congestion_level']  # Target variable is 'congestion_level' column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
rf_regressor = RandomForestClassifier(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)

In [9]:
# Print confusion matrix and classification report with corresponding labels
print("\nConfusion Matrix:")
print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=class_labels, columns=class_labels))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_labels))


Confusion Matrix:
                     Low Congestion  Moderate Congestion  High Congestion
Low Congestion                14446                   10                0
Moderate Congestion              14                 9578               95
High Congestion                   1                   99             1345

Classification Report:
                     precision    recall  f1-score   support

     Low Congestion       1.00      1.00      1.00     14456
Moderate Congestion       0.99      0.99      0.99      9687
    High Congestion       0.93      0.93      0.93      1445

           accuracy                           0.99     25588
          macro avg       0.97      0.97      0.97     25588
       weighted avg       0.99      0.99      0.99     25588



In [7]:
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_train, y_train)
y_pred = knn_classifier.predict(X_test)

In [8]:
# Print confusion matrix and classification report with corresponding labels
print("\nConfusion Matrix:")
print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=class_labels, columns=class_labels))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_labels))


Confusion Matrix:
                     High Congestion  Low Congestion  Moderate Congestion
High Congestion                   32              16                   56
Low Congestion                    41            1076                  242
Moderate Congestion               60             249                  535

Classification Report:
                     precision    recall  f1-score   support

    High Congestion       0.24      0.31      0.27       104
     Low Congestion       0.80      0.79      0.80      1359
Moderate Congestion       0.64      0.63      0.64       844

           accuracy                           0.71      2307
          macro avg       0.56      0.58      0.57      2307
       weighted avg       0.72      0.71      0.72      2307



In [7]:
class_labels = ['Low Congestion', 'Moderate Congestion', 'High Congestion']
# Initialize XGBoost classifier
xgb_classifier = xgb.XGBClassifier()

# Train the classifier
xgb_classifier.fit(X_train, y_train)

# Predictions on the testing set
y_pred = xgb_classifier.predict(X_test)
# Print confusion matrix and classification report with corresponding labels

print("\nConfusion Matrix:")
print(pd.DataFrame(confusion_matrix(y_test, y_pred), index=class_labels, columns=class_labels))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_labels))


Confusion Matrix:
                     Low Congestion  Moderate Congestion  High Congestion
Low Congestion                14442                   14                0
Moderate Congestion               7                 9596               84
High Congestion                   0                  103             1342

Classification Report:
                     precision    recall  f1-score   support

     Low Congestion       1.00      1.00      1.00     14456
Moderate Congestion       0.99      0.99      0.99      9687
    High Congestion       0.94      0.93      0.93      1445

           accuracy                           0.99     25588
          macro avg       0.98      0.97      0.97     25588
       weighted avg       0.99      0.99      0.99     25588



In [11]:
# Export model
pickle.dump(xgb_classifier, open('model_XGB_1.pkl', 'wb'))
# Read the Model
#model_pre_trained = pickle.load(open('model_XGB.pkl','rb'))

In [18]:
features = X_test.iloc[0].values

# Create a DataFrame with the extracted features
test_df = pd.DataFrame([features], columns=X_test.columns)

# Make predictions
prediction = model_pre_trained.predict(test_df)
prediction.tolist()[0]

1